In [1]:
import numpy as np
from scipy.signal import butter,filtfilt
import matplotlib.pyplot as plt
import pyxdf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import math #math fun
import matplotlib.pyplot as plt #plotting
from mpl_toolkits.mplot3d import Axes3D #3d plots
from sklearn.neighbors import NearestNeighbors 

from  matplotlib.ticker import FuncFormatter

def greaterThanThis(arr,number):
    s=arr>number
    s=np.squeeze(s)
    index=np.squeeze(np.where(s==True))
    return index[0]

def lslSynchronize(streams):
    for i in range(np.shape(streams)[0]):  #### np.shape(streams)[0] for more streams 
        if streams[i]['time_stamps'] != []:
            streams[i]['time_stamps']=streams[i]['time_stamps']-min(streams[i]['time_stamps'])
    return streams

def butterBandpass(signal, lowcut, highcut, fs, order):
    nyq=0.5*fs
    low = lowcut/nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    filtered = filtfilt(b, a, signal)
    # print(filtered)
    return filtered

def plotSpectrum(signal, fs):
    n=len(signal)
    k=np.arange(n)
    time=n/fs
    frequency_spectra=k/time
    frequency_spectra=frequency_spectra[range(int(n/2))]
    Y=np.fft.fft(signal)/n
    Y=Y[range(int(n/2))]

    plt.subplot(2,1,1)
    plt.plot(k/fs,signal,'b')
    plt.xlabel(' time(s)')
    plt.title('Time domain visualisation')

    plt.subplot(2,1,2)
    plt.plot(frequency_spectra,abs(Y),'r')
    plt.xlabel( 'Freq (Hz) ')
    plt.title('Frequency domain visualisation')

class dataOrg():
    def __init__(self):
        self.type="extracted from XDF files"
    def extractData(self,streams,string):
        n_s=np.shape(streams)[0]
        typeStream=[]
        for i in range(n_s):
            tempStream=streams[i]
            typeStream.append(tempStream['info']['type'][0])
        index=typeStream.index(string)
        currStream=streams[index]
        self.type=string
        self.data=currStream['time_series']
        # print(currStream['time_series'])
        self.timeStamps=currStream['time_stamps']
        return self

def filterAll(data):
    data=np.transpose(data)
    data=data.astype(float)
    filteredData=[]
    for i in range(np.shape(data)[0]):
        filt_data=butterBandpass(data[i,:],1,35,256,5)
        # print(filt_data)
        filteredData.append(filt_data)
    return np.array(filteredData)

def prepare3D(eegData,eegTimestamps,markersData,markersTimestamps):
#     print(eegData.sha)
    # count=0
    training_data=[]
    training_markers=[]
    for i in range(len(markersTimestamps)):
        
        index=greaterThanThis(eegTimestamps,markersTimestamps[i])
        training_data.append(eegData[:,index:index+256])
        
        training_markers.append(markersData[i])
#         print(i, len(training_data),len(training_markers))
    training_data=np.array(training_data)
    training_markers=np.array(training_markers)
    return training_data,training_markers


def extractTrain(filenames):
    flag=0
    for fname in filenames:
        flag+=1
        
        streams,fileheader=pyxdf.load_xdf(fname)
#         for i in range(np.shape(streams)[0]):
#             print(i,streams[i]['info'])
        streams=lslSynchronize(streams)
        stream = streams[4]
        nb_chan = int(stream['info']['channel_count'][0])
        channels = [(stream['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chan)]
        print(channels)
        eeg=dataOrg().extractData(streams,'EEG')
        markers=dataOrg().extractData(streams,'Markers')
        markers.data=np.array(markers.data)
        print('XDF - raw')
        print(markers.data.shape)
        print(eeg.data.shape)
#         eeg.data = eeglab_raw['data'][0].T[:,:8].T
        eeg.data=filterAll(eeg.data[:,0:4])
        print('Filtered shape')
        print(eeg.data.shape)

        if flag==1:
            training_data,training_markers=prepare3D(eeg.data,eeg.timeStamps,markers.data,markers.timeStamps)
            continue
        temp_data,temp_markers=prepare3D(eeg.data,eeg.timeStamps,markers.data,markers.timeStamps)
        training_data=np.concatenate((training_data,temp_data),axis=0)
        training_markers=np.concatenate((training_markers,temp_markers),axis=0)
    return training_data,training_markers


In [2]:
### target is 0, non-target is 1

def get_Xy(filenames,markers_row,markers_column):

    data,markers = extractTrain(filenames)
    print(data.shape)
    print('**************************************')
    channels= [0,1,2,3]
    data_all = []
    for channel in channels:

        data_all.append(data[:,channel,:])
    ind1 = np.where(markers==markers_row)[0]
    ind2 = np.where(markers==markers_column)[0]


    ind = list(np.arange(markers.shape[0]))
    for i in ind1:
        ind.remove(i)
    for i in ind2:
        ind.remove(i)

    labels = np.zeros(len(markers))
    labels[ind] = 1
    labels[ind1] = 0
    labels[ind2] = 0
    labels = np.expand_dims(labels,-1)
    return np.array(data_all),labels
    

## xdf files


In [3]:
import mne

# filenames=[r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided.xdf",r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided_old8.xdf",r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided_old7.xdf",r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided_old6.xdf",r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided_old5.xdf"]
filenames=[r"C:\Users\Vinay\Downloads\setup_files\LabRecorder-1.16.2-Win_amd64\LabRecorder\rishi_A.xdf"]
#            r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\wheelchair\P300_paradigm\Muse\data\exp001\P003_block_MemoryGuided_old1.xdf" ] ##### q is 03, 14 
# print(filenames[4])
X,y = get_Xy(filenames,'03','14')

print(X.shape,y.shape)


C:\Users\Vinay\.conda\envs\eeg-notebooks\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


IndexError: list index out of range

In [ ]:
features=X.transpose(1, 0, 2)
labels=np.array(y)
print(features.shape)
v = features[np.where(labels==0)[0]]
v_n = features[np.where(labels==1)[0]]

print(v.shape,v_n.shape)
 
for i in range(4):

    plt.figure()
    plt.plot(np.mean(v[:,i,:],axis=0),'r')
    plt.plot(np.mean(v_n[:,i,:],axis=0),'b')
#     plt.ylim([-10,10])
    plt.gca().get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x*1000/256), ',')))
    plt.show()


In [ ]:
v.shape

In [ ]:
v_n.shape

In [ ]:
v_temporal = v[:,[0,-1],:]
v_n_temporal = v_n[:,[0,-1],:]

plt.figure()
plt.plot(np.mean(v_temporal,axis=(0,1)),'r')
plt.plot(np.mean(v_n_temporal,axis=(0,1)),'b')
plt.gca().get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x*1000/256), ',')))
    
plt.show()

In [ ]:
def read_data():
    streams,fileheader=pyxdf.load_xdf(r"C:\Users\Vinay\OneDrive - IIT Kanpur\Documents\LabRecorder\foo.xdf")
    stream = streams[0]
    nb_chan = int(stream['info']['channel_count'][0])
    channels = [(stream['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chan)]
    df = pd.DataFrame(stream['time_series'], stream['time_stamps'], channels)

    d = df.to_numpy()
    d = np.transpose(d)
    data = d

    return data,df

In [ ]:
v_n.shape

In [ ]:
### downsampling
### plot psd and do sanity check
### ASR cleaning
### Common average referencing
eeg_data = eeg_data / 1000000  # BrainFlow returns uV, convert to V for MNE

# Creating MNE objects from brainflow data arrays
ch_types = ['eeg'] * len(eeg_channels)
ch_names =  self.getChannelNames('eeg')
sfreq = BoardShim.get_sampling_rate(self.brainflow_id,0) # 0 for default preset i.e. EEG 
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(eeg_data, info)